Kurset gennemgår grundlæggende Python-kode, der kan få dig i gang med at bruge programmering som redskab til tekstbehandling, kvantitative analyser og tekst- og datamining.

Mere teknisk fortalt gennemgår vi begreber som variabler, værdier, datatyperne tekststrenge, lister og loops. 

Vi gennemgår et eksempel på, hvordan man kan hente tekstdata, klargøre data og bruge biblioteket jieba. Jieba anvendes til at opdele tekst i ord og undersøtter traditionelt kinesisk. 

_Kilde: https://github.com/fxsjy/jieba_

# Import af biblioteker

In [ ]:
# Til navigation på computerne
import os

# Webscrape biblioteker
from bs4 import BeautifulSoup
import requests

# Til klargøring og analyse
import jieba
import nltk
import re

Vi laver en variabel, som vi bruger til at gemme url'en til den sider, som vi vil webscrabe. 

Vi skal scrape denne wikipidiaside: 反对逃犯条例修订草案运动

In [ ]:
# gem url'en i en variabel
url_zh = 'https://zh.wikipedia.org/zh-cn/%E5%8F%8D%E5%B0%8D%E9%80%83%E7%8A%AF%E6%A2%9D%E4%BE%8B%E4%BF%AE%E8%A8%82%E8%8D%89%E6%A1%88%E9%81%8B%E5%8B%95'

Vi indsætter et af variablenavnene i request.get('url') nedenfor.

In [ ]:
# hent data
page = requests.get(url_zh)

# scrape websiden
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
# find alle 'headline3 og paragraph-tags'
tags = soup.find_all(['h1', 'h3', 'p'])

# læs teksten ud af p_tags og 'join' den returnerede liste i variablen 'text'
text = ' '.join([p.get_text() for p in tags]).strip()

In [ ]:
print (text)

# Klargøring af tekst 

## Rensning

Teksten bestå af latinske bogstaver og kinesiske tegn.

Ønsker man at sortere de latiske bogstaver fra kan man bruge koden nedenfor.

_Kilder:_
    
_https://stackoverflow.com/questions/2718196/find-all-chinese-text-in-a-string-using-python-and-regex_

_https://unicode-table.com/en/blocks/cjk-unified-ideographs/_

In [ ]:
chinese_list = re.findall(r'[\u4e00-\u9fff]+', text)

In [ ]:
print (chinese_list)

Listen kan man efterfølgende samle til en tekststreng igen med .join()

_Kilde: "https://www.w3schools.com/python/ref_string_join.asp"_   

In [ ]:
chinese_text = ' '.join(chinese_list)

In [ ]:
print (chinese_text)

## Tekstdeling

I jieba.lcut metoden indsætter vi en tekststreng eller, som i dette tilfælde, en variabel som indeholder en tekststreng, og vi styrer cut-tilstanden. L'et i .lcut() henviser til at metoden returerer en liste. 'Cut_all=True' skulle give flest mulige orddelinger af teksten, være hurtig, men mindre præcis. 'Cut_all=False' skulle være mere nøjagtige end den første, og dermed mere velegnet til tekstanalyse. _Kilde: https://github.com/fxsjy/jieba_

In [ ]:
seg_list1 = jieba.lcut(chinese_text, cut_all=False)

In [ ]:
seg_list1

Vi får returneret mange felter, der består af 'white_space'.
For at så disse linjer fjernet fra vores data bruger vi 'if' til at sætte en betingelse ind i koden. Vi skriver, hvis vores linjer består af tegn, som ikke er lig med 'white space', så er vi interesseret i at gemme det i vaiablen seg_list2. 

In [ ]:
seg_list2 = [i for i in seg_list1 if i != ' ']

In [ ]:
print (seg_list2)

Som det fremgår ovenfor bliver lister lavet vha. firkantede parenteser ( [ ] ).

Man kan tilgå elementerne i listen ved at referere til indekstallet. Igen kan vi bruge både positive og nagative tal. Husk at i Python er første indextal 0 og ikke 1, hvilket betyder, at vi tilgår det første og det sidste element i listen på denne måde:

In [ ]:
print (seg_list2[0])
print (seg_list2[-1])

# Part of Speech Tagging (POS)

Jiebas part of speech tagger returnerer ordene og tags i to forskellige elementer. For at bruge pos-taggeren skal man importere 'import jieba.posseg as pseg'.

Ifølge dokumentationen bruger man pseg efterfuldt af .cut( 'tekst_streng' ).
_Kilde: "4. Part of Speech Tagging https://github.com/fxsjy/jieba"_

Vi får returneret ord og tags. De ligger i .word og .flag. I dokumentationen viser programmøren, hvordan man printer ord og tags, men jeg vil gerne have alle ord og tags gemt som par i en liste. Derfor bruger jeg en tuple, som er en python datatype og tilføjer hvert ord og tag par til en liste, som jeg kalder for 'pos'.
_Kilde: Python Tuples https://www.w3schools.com/python/python_tuples.asp_

In [ ]:
import jieba.posseg as pseg
words = pseg.cut(chinese_text)
pos_tags = []
for w in words:
    if w.word > ' ':
        word_tag = tuple((w.word, w.flag))
        pos_tags.append(word_tag)

In [ ]:
pos_tags

Jeg skriver et for loop, der indeholder en betingelse ('if'). Med loopet gennemgår jeg listen med tuples. Hvis første element i 'tuplen' ([1]) er lig med 'v' tilføjer jeg første element ([0]) til listen 'words'.   

In [ ]:
words = []
for item in pos_tags:
    if item[1] == 'v':
        words.append(item[0])

In [ ]:
words 

Før vi ser på fordelingen af ord med 'v'-tags, er vi nødt til at gøre noget ved at python som default ikke kan printet kinesiske tegn. Derfor importerer vi 'matplotlib.pyplot as plt' og ændrer font.family til "Microsoft YaHei".

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Microsoft YaHei"

Herefter kan vi importere nltk og anvende nltk.FreqDist().

In [ ]:
import nltk
nltk.FreqDist(pos_tags).plot(20)

#### Opgave: prøv at udskifte 'v' med andre tags.

## Stopord

Stopord er småord, som ofte ikke er betydningsbærende ord.

Vi har defor brug for at indlæse en stopordsliste. Den ligger i mappen stopwords, og den er fundet på jiebas github side: https://github.com/fxsjy/jieba/tree/master/extra_dict.

Vi bruger os biblioteket til at navigere over i stopordsmappen. Hver kan det blive lidt indviklet, fordi når det kommer til at navigere mellem mapper, så er der en lille forskel på om, man sidder med en mac eller en pc.

Jeg sidder med en pc og skal bruge '\\' som separator i min sti til min mappe, men hvis du har en  mac, så skal du bruge '/' som din separator i i stedet. 

In [ ]:
# find din separator
os.sep

In [ ]:
# find din nuværende mappe
os.getcwd()

In [ ]:
# find ud af, hvad der ligger i mappen
os.listdir()

In [ ]:
# flyt dig hen i mappen med stopord
os.chdir('.\\stopwords')

In [ ]:
# find din nuværende mappe igen
os.getcwd()

In [ ]:
# find ud af, hvad der ligger i mappen - igen
os.listdir()

Vi er klar til at indlæse vores stopord.

In [ ]:
sw_ch = open('stopwords_ch.txt', 'r', encoding='utf-8-sig').read().split()

In [ ]:
sw_ch

Nu kan alle teksterne bliver filtreret for stopord.

In [ ]:
filtered_tokens = []
for word in seg_list2:
    if word not in sw_ch:
        filtered_tokens.append(word)

Med en ny ordliste, der ikke længere indeholder stopord, kan vi få overblik over, hvilke betydningsbærende ord, der flyder mest.

In [ ]:
fdist_filtered = nltk.FreqDist(filtered_tokens).plot(20, title='Hyppigste ord (uden stopord)')

In [ ]:
long_tokens = []

for word in filtered_tokens:
    if len(word) > 4:
        long_tokens.append(word)

In [ ]:
fdist_filtered = nltk.FreqDist(long_tokens).plot(20, title='Længste ord')

# NLTK metoder

Jeg har benyttet nltk mange gange, men aldrig med kinesisk tekst. Vi eksperimenterer og opretter et nltk-tekstobjekt, som burde give os mulighed for at anvende forskellige nltk-metoder.

In [ ]:
nltk_text = nltk.Text(seg_list2)

collocation_list() returnerer en liste over de mest almindelige ordpar i teksten. Bemærk, at i nogle versioner af Python virker collocation_list() ikke. Hvis dette er tilfældet, prøv _collocations()_ i stedet.

In [ ]:
nltk_text.collocation_list()

Concordance()-metoden returnerer konteksten af et specifikt udtryk. Længden af output kan ændres med parametrene i width og lines.

In [ ]:
nltk_text.concordance('和', lines=30, width=40)

For at identificere ord, der optræder i en lignende kontekst, kan vi bruge metoden similar().

Jeg har en forestilling om at metoden giver bedre resultater jo længere teksten er.

In [ ]:
nltk_text.similar('和')

generate() metoden kan du bruge til at genere mere eller mindre sammenhængende tekst med udgangspunkt i en eksisterende tekst.

In [ ]:
text_gen = nltk_text.generate(150)